# Описание

Это архивный ноутбук с описанием создания и тестов annoy и KNN для подбора ответов болталки. Его стоит смотреть во вторую очередь.
- [Главный ноутбук](./../chat_bot.ipynb)

# Cписок зависимостей
- python==3.11.1

```
annoy==1.17.1 anyio==3.6.2 argon2-cffi==21.3.0 argon2-cffi-bindings==21.2.0 arrow==1.2.3 asttokens==2.2.1 attrs==22.2.0 Babel==2.11.0 backcall==0.2.0 beautifulsoup4==4.11.1 bleach==5.0.1 certifi==2022.12.7 cffi==1.15.1 charset-normalizer==2.1.1 click==8.1.3 colorama==0.4.6 comm==0.1.2 contourpy==1.0.6 cycler==0.11.0 Cython==0.29.32 debugpy==1.6.4 decorator==5.1.1 defusedxml==0.7.1 entrypoints==0.4 executing==1.2.0 fastjsonschema==2.16.2 fonttools==4.38.0 fqdn==1.5.1 fst-pso==1.8.1 FuzzyTM==2.0.5 gensim==4.3.0 idna==3.4 ipykernel==6.19.4 ipython==8.8.0 ipython-genutils==0.2.0 isoduration==20.11.0 jedi==0.18.2 Jinja2==3.1.2 joblib==1.2.0 json5==0.9.11 jsonpointer==2.3 jsonschema==4.17.3 jupyter-events==0.5.0 jupyter_client==7.4.8 jupyter_core==5.1.2 jupyter_server==2.0.6 jupyter_server_terminals==0.4.3 jupyterlab==3.5.2 jupyterlab-pygments==0.2.2 jupyterlab_server==2.18.0 kiwisolver==1.4.4 lxml==4.9.2 MarkupSafe==2.1.1 matplotlib==3.6.2 matplotlib-inline==0.1.6 miniful==0.0.6 mistune==2.0.4 nbclassic==0.4.8 nbclient==0.7.2 nbconvert==7.2.7 nbformat==5.7.1 nest-asyncio==1.5.6 nltk==3.8.1 notebook==6.5.2 notebook_shim==0.2.2 numpy==1.24.1 packaging==22.0 pandas==1.5.2 pandocfilters==1.5.0 parso==0.8.3 patsy==0.5.3 pickleshare==0.7.5 Pillow==9.4.0 platformdirs==2.6.2 prometheus-client==0.15.0 prompt-toolkit==3.0.36 protobuf==4.21.12 psutil==5.9.4 pure-eval==0.2.2 pycparser==2.21 pyFUME==0.2.25 Pygments==2.14.0 pymystem3==0.2.0 pyparsing==3.0.9 pyrsistent==0.19.3 python-dateutil==2.8.2 python-json-logger==2.0.4 pytz==2022.7 pywin32==305 pywinpty==2.0.10 PyYAML==6.0 pyzmq==24.0.1 regex==2022.10.31 requests==2.28.1 rfc3339-validator==0.1.4 rfc3986-validator==0.1.1 scikit-learn==1.2.0 scipy==1.10.0 Send2Trash==1.8.0 simpful==2.9.0 six==1.16.0 smart-open==6.3.0 sniffio==1.3.0 soupsieve==2.3.2.post1 stack-data==0.6.2 statsmodels==0.13.5 terminado==0.17.1 threadpoolctl==3.1.0 tinycss2==1.2.1 tomli==2.0.1 tornado==6.2 tqdm==4.64.1 traitlets==5.8.0 uri-template==1.2.0 urllib3==1.26.13 wcwidth==0.2.5 webcolors==1.12 webencodings==0.5.1 websocket-client==1.4.2
```

# Импорт и функции

In [19]:
### standard ###
import itertools
import json 
import re
import numpy as np
from joblib import Parallel, delayed


#### site-packages ###
import pandas as pd
from pymystem3 import Mystem
from nltk.corpus import stopwords
import gensim
import pickle
from annoy import AnnoyIndex

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, preprocessing
from sklearn import linear_model, metrics
from sklearn.neighbors import KNeighborsClassifier

'''Class for properties column preprocessing'''
#‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾#
class properties_prerpoc():
    def __init__(self, ser):
        self.ser = ser

    def show_ser(self):
        return self.ser
        
    '''Get dictionaries from suitable string  '''
    def get_dict(string):
        try: result = eval(string) # .values()
        except: result = None # urls also will be here
        
        return result
    
    '''Get series from prepared Series with dictionaries''' ###############################################DFDSFSDFSDFSFSDFSDFSDFSDFSDFSD
    def get_series_of_dicts(self):
        for i in self.ser.index:
            self.ser.at[i] = get_dict(self.ser[i])
    
    '''Get all keys and values from prepared Series with dictionaries'''
    def get_all_keys_and_values(self):
        clean_properties = self.ser.dropna()  #

        clean_properties_keys = []
        clean_properties_values = []

        for dct in clean_properties.values:
            clean_properties_keys += dct.keys()
            clean_properties_values += dct.values()

        clean_properties_keys = pd.Series(clean_properties_keys)
        clean_properties_values = pd.Series(clean_properties_values)
        
        return clean_properties_keys,clean_properties_values
    
    ''' Make Series with lists of keys and values 
        from prepared Series with dictionaries    '''
    def compress_expand_lists_to_ser(self):
        properties_key_lists = self.ser.copy()
        properties_value_lists = self.ser.copy()

        for i in self.ser.index:
            try:
                properties_key_lists[i] = list(properties_key_lists[i].keys())
                properties_value_lists[i] = list(properties_value_lists[i].values())
            except:
                properties_key_lists[i] = None
                properties_value_lists[i] = None
            
        return properties_key_lists, properties_value_lists
    
    '''Replace string "duno" with ["duno"]''' ##################################### Не робит, ибо не dunno, а ненаю
    def replace_dunno_with_list(self, column):
        for i in column.index:
            name = column.name
            if df[name][i] == "dunno":
                df.loc[:, name][i] = ["dunno"]



#‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾#
'''Get dictionaries from suitable string  '''
def get_dict(string):
    try: result = eval(string) # .values()
    except: result = None # urls also will be here
    return result
#___________________________________________________#



"""Class for preprocessing especially for model""" ### Удалена обработка знаков препинания
#‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾#
class model_prerpoc():
    def __init__(self, ser):
        self.ser = ser
    
    '''Transforms series to one string without stop-words and garbage symbols'''
    def prepearing_for_lemmatization(self):
        series = self.ser
        
        if series.dtype == object:
            
            ## making one big string
            series = series.apply(lambda x: str(x)) ##!!!
            union_string = series.str.cat(sep="`<>`")
            
            return union_string
        
        # union descrirption into one string
        union_string = series.str.cat(sep="`<>`")
        
        return union_string

    '''lemmatization with pymystem3 and parallelizm'''
    def lemmatization(self, text_for_mystem):
        # split large string to 6 lists with smaller strings for parallel lemmatization
        arrays = pd.Series(np.array_split(text_for_mystem.split('`<>`'), 6))

        ## concatenate strings in arrays with separator for future split
        union_string_6 = []
        for array in arrays:
            union_string_6.append(pd.Series(array).str.cat(sep="`<>`"))


        ## here is parallel lemmatization
        m = Mystem().lemmatize
        results = Parallel(n_jobs=-1)(delayed(m)(x) for x in union_string_6)

        ## replace pointless tabulation
        strings = []
        for array in results:
            text = " ".join(array).replace("   ", " ").replace("  ", " ")
            text = re.sub("не\s", "не", text) ### !  join "не"
            strings.append(text)

        ## split strings to arrays
        lemma_array = []
        for string in strings:
            lemma_array.append(string.split("`<>`"))

        output = []
        for lst in lemma_array:
            for string in lst:
                output.append(string)

        lemma_series = pd.Series(output)
        lemma_series = lemma_series.astype("string")
        return lemma_series

"""all annoy need to be a bit more simple"""
class annoy:
    
    def __init__(self, model_answers, model):
        self.model = model
    
    def predict(self, vector):
        return df.answer.iloc[u.get_nns_by_vector(vector, 1)].values[0]
        

def give_to_model(string, w2v, model):
    text = " ".join((Mystem().lemmatize(string))) # leammatization

    lemmatized_words = gensim.utils.simple_preprocess(text)     # word preprocessing

    lst = []
    for i, word in enumerate(lemmatized_words):
            try:lst.append( w2v.wv.get_vector(word) )
            except KeyError: pass
        
    vector = np.sum(np.array(lst), axis=0)
    
    if "KNeighborsClassifier" in str(model):
        return (model.predict([vector])[0])
    
    if "annoy" in str(model):
        return model.predict(vector)

## Обработка большого датасета

In [2]:
"""Get one answer for each question from big dataset"""
topics = []

with open("Otvety.txt", "r", encoding="utf-8") as f:
    next(f)
    co = -1
    for i, row in enumerate(f):
        if row == "---\n": 
            topics.append([])
            co += 1
        else:
            topics[co].append(row)

questions = []
answers   = []

for lst in topics:
    questions.append(lst[0])
    answers.append(lst[1])
    
qa = pd.Series(questions, answers)

In [3]:
## 16 seconds. Getting Series to work with.
qa = pd.Series(questions, answers)
qa.to_csv('qa.csv', index=True, header=["question"], mode="w")

In [5]:
del topics
del qa
del questions
del answers

## Лемматизация
Выполненна по чанкам, ибо операции очень затратны для оперативной памяти.

In [7]:
%%time
### get needed columns in chunks from big dataset for memory safety reason
data = pd.read_csv("qa.csv", chunksize=40000, dtype=str)

i = 0 
for chank in data:
        
    q_piece = chank.question.str.lower().apply\
        (gensim.utils.simple_preprocess).apply\
        (lambda x: " ".join(x))

    a_piece = chank["Unnamed: 0"].reset_index(drop=True)
    qa_preproc = model_prerpoc(q_piece)
    qa_text = qa_preproc.prepearing_for_lemmatization()
    piece_lemma = qa_preproc.lemmatization(qa_text)

    qa_df = pd.concat([a_piece, piece_lemma],ignore_index=True, axis=1)
    qa_df.to_csv('qa_lemma.csv', mode="a", index=False, header=False)


CPU times: total: 1min 35s
Wall time: 6min 4s


## Word2Vec

In [32]:
df = pd.read_csv('qa_lemma.csv', header=None)
df.columns = ["answer", "question"]
df = df.astype("string")
df.dropna(inplace=True)
df.reset_index(inplace=True,drop=True)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161698 entries, 0 to 1161697
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   answer    1161698 non-null  string
 1   question  1161698 non-null  string
dtypes: string(2)
memory usage: 17.7 MB


In [36]:
# ten minutes
model = gensim.models.Word2Vec(sentences=df.question,
                               vector_size=300,
                               window=5,
                               min_count=1,
                               workers=4,
                               epochs=40)

In [37]:
### save the model
model.save("w2v_model_qa.model")

## Векторизация датасета

In [1]:
df = pd.read_csv('qa_lemma.csv', header=None)
df.columns = ["answer", "question"]
df = df.astype("string")

df.dropna(inplace=True)
df.reset_index(inplace=True,drop=True)
wrong_rows = [132645, 144364, 431016, 485648, 571362, 624526, 646750, 755784, 839713]
df.drop(wrong_rows, inplace=True) #drop rows without question
df.reset_index(inplace=True,drop=True)

df["question"] = df.question.apply(gensim.utils.simple_preprocess)

model = gensim.models.Word2Vec.load("w2v_model_qa.model")

In [2]:
lst_of_the_arrays = []
wrong_rows = []

for idx, row in enumerate(df.question):
    lst = []
    for word in row:
        lst.append(model.wv.get_vector(word))
        
    try:
        lst_of_the_arrays.append(np.sum(lst, axis=0))
    except:
        lst_of_the_arrays.append(None)
        wrong_rows.append(idx)
        

In [3]:
df_arrays = pd.DataFrame(lst_of_the_arrays)

In [18]:
u_df = pd.concat([df, df_arrays], axis=1)

In [14]:
len(df), len(df_arrays)

(1161689, 1161689)

In [17]:
df.reset_index(inplace=True,drop=True)

In [19]:
u_df

,answer,question,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,хомячка....,"[вопрос, тдв, давно, хорошо, отдыхать, лично, ...",-1.735446,3.335729,-0.603580,-4.140172,-4.046233,-5.413363,1.464721,11.956296,...,-10.192062,-3.959229,-16.461716,1.048521,4.593544,-12.359629,9.029293,6.567714,-9.199703,-1.085837
1,меня вобще прикалывает эта тема :).,"[как, парень, относиться, цветной, линза, если...",-3.626490,6.690380,-10.050926,-3.927771,-17.246191,-3.441206,8.829128,6.829137,...,-12.650711,15.693406,-1.885282,-1.693082,12.139981,-1.375306,10.089205,10.456690,-6.659889,-12.238979
2,"Если это ""счастье "" действительно на вас свали...","[что, делать, сегодня, находить, миллион, рубль]",4.589223,7.705190,-1.960143,-11.165007,-7.590022,-1.047975,-1.670039,5.502703,...,-4.189414,2.917729,-5.197858,1.193294,4.528908,0.134092,-0.636302,9.839035,-6.932865,-5.893131
3,ЭБУ — электронный блок управления двигателем а...,"[эбу, двенашка, называться, итэлма, что, за, эбу]",1.283373,-1.361773,-0.330553,-3.724918,-4.689672,6.050868,-2.367335,5.141415,...,-0.977934,4.665846,0.243961,2.050638,1.705771,0.271662,-1.439371,2.739101,1.064514,0.238557
4,"4. Охотники и Жертвы, Ледяной укус, Поцелуй ть...","[академия, вампир, сколько, на, даный, момент,...",7.537388,3.443851,-4.579748,4.645518,-14.197975,-2.892067,-6.302214,2.403178,...,-7.374341,7.507833,2.762842,8.664568,-5.016368,8.288232,-7.605923,2.631406,-7.616882,-2.665820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161684,"Конечно, есть. Оба эти слова пришли из латыни,...","[между, слово, президент, резидент, быть, что,...",-4.726631,-3.024945,-0.470489,-3.825744,-3.838722,-6.061239,8.276185,3.829201,...,-1.200712,8.313508,-7.236314,8.767304,-1.613073,-1.970937,3.411723,5.589098,-8.533741,1.132317
1161685,"Если плохо обращаться со СВОЕЙ собакой, она сд...","[если, это, мой, то, оно, никуда, от, недеваться]",3.340455,10.122863,-7.271936,-4.150289,-12.597342,-1.710233,6.341134,8.559175,...,3.359941,15.091386,-0.228072,-1.139926,5.022292,-1.075924,0.923358,17.735033,-7.407753,-7.436988
1161686,"Люблю, но нет её..","[вы, халява, любить, или, совесть, иметь]",0.563658,2.844564,-1.421358,-3.796564,-2.620279,-0.796618,9.095448,2.697136,...,1.090251,7.214147,-9.474057,-3.616992,4.732539,-9.893394,4.481746,4.028037,-2.910013,-1.683667
1161687,[ссылка заблокирована по решению администрации...,"[так, много, разный, гороскоп, кто, нибудь, мо...",7.180774,15.200850,-15.890986,-11.314883,-6.492365,-18.555244,11.946363,15.163485,...,-12.815183,13.974255,-12.880548,6.933138,-3.057068,-28.168142,-1.613786,27.903208,-20.898998,-2.571362


In [4]:
df_arrays

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-1.735446,3.335729,-0.603580,-4.140172,-4.046233,-5.413363,1.464721,11.956296,2.506700,0.761430,...,-10.192062,-3.959229,-16.461716,1.048521,4.593544,-12.359629,9.029293,6.567714,-9.199703,-1.085837
1,-3.626490,6.690380,-10.050926,-3.927771,-17.246191,-3.441206,8.829128,6.829137,3.100150,-3.185428,...,-12.650711,15.693406,-1.885282,-1.693082,12.139981,-1.375306,10.089205,10.456690,-6.659889,-12.238979
2,4.589223,7.705190,-1.960143,-11.165007,-7.590022,-1.047975,-1.670039,5.502703,1.689579,-1.929160,...,-4.189414,2.917729,-5.197858,1.193294,4.528908,0.134092,-0.636302,9.839035,-6.932865,-5.893131
3,1.283373,-1.361773,-0.330553,-3.724918,-4.689672,6.050868,-2.367335,5.141415,6.879317,3.413110,...,-0.977934,4.665846,0.243961,2.050638,1.705771,0.271662,-1.439371,2.739101,1.064514,0.238557
4,7.537388,3.443851,-4.579748,4.645518,-14.197975,-2.892067,-6.302214,2.403178,1.540559,-3.829041,...,-7.374341,7.507833,2.762842,8.664568,-5.016368,8.288232,-7.605923,2.631406,-7.616882,-2.665820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161684,-4.726631,-3.024945,-0.470489,-3.825744,-3.838722,-6.061239,8.276185,3.829201,-5.273933,2.431054,...,-1.200712,8.313508,-7.236314,8.767304,-1.613073,-1.970937,3.411723,5.589098,-8.533741,1.132317
1161685,3.340455,10.122863,-7.271936,-4.150289,-12.597342,-1.710233,6.341134,8.559175,0.111143,-3.824245,...,3.359941,15.091386,-0.228072,-1.139926,5.022292,-1.075924,0.923358,17.735033,-7.407753,-7.436988
1161686,0.563658,2.844564,-1.421358,-3.796564,-2.620279,-0.796618,9.095448,2.697136,-2.876876,-3.107685,...,1.090251,7.214147,-9.474057,-3.616992,4.732539,-9.893394,4.481746,4.028037,-2.910013,-1.683667
1161687,7.180774,15.200850,-15.890986,-11.314883,-6.492365,-18.555244,11.946363,15.163485,-10.460518,-13.741971,...,-12.815183,13.974255,-12.880548,6.933138,-3.057068,-28.168142,-1.613786,27.903208,-20.898998,-2.571362


In [67]:
outfile = open("qa.pcl",'wb')

pickle.dump(df, outfile)
outfile.close()

In [20]:
outfile = open("qa_preproc_df.pcl",'wb')

pickle.dump(u_df, outfile)
outfile.close()

In [2]:
df = pickle.load( open("qa_matrix_df.pcl",'rb') )
df["answer"] = df.answer.str.replace("<br>", "\n")

In [ ]:
X = df.drop(["answer", "question"], axis="columns")

In [22]:
neigh = KNeighborsClassifier(n_neighbors=8).fit(X, df.answer)

In [ ]:
for i, word in enumerate(lemmatized_words):
    try:lst.append( model.wv.get_vector(word) )
    except KeyError: pass

# Annoy

In [ ]:
w2v = gensim.models.Word2Vec.load("w2v_model_qa.model")

In [15]:
annoy_model = annoy(df.answer, annoy)

In [2]:
df = pickle.load( open("qa_matrix_df.pcl",'rb') )
X = df.drop(["answer", "question"], axis="columns")

In [5]:
df.answer

0                                              хомячка.... 

1                      меня вобще прикалывает эта тема :). 

2          Если это "счастье " действительно на вас свали...
3          ЭБУ — электронный блок управления двигателем а...
4          4. Охотники и Жертвы, Ледяной укус, Поцелуй ть...
                                 ...                        
1161684    Конечно, есть. Оба эти слова пришли из латыни,...
1161685    Если плохо обращаться со СВОЕЙ собакой, она сд...
1161686                                 Люблю, но нет её.. 

1161687    [ссылка заблокирована по решению администрации...
1161688    Ничего, дорогого Леонида Ильича слушали, и теб...
Name: answer, Length: 1161689, dtype: string

In [4]:
chat_answers = pickle.load( open("answers_for_annoy.pcl",'rb') )

EOFError: Ran out of input

In [4]:
X = df.drop(["answer", "question"], axis="columns")

In [27]:
f = len(X.iloc[0])
t = AnnoyIndex(f, 'angular')

for i in range(len(X)):
    t.add_item(i, list(X.iloc[i]))
    
t.build(100)


True

In [ ]:
t.save('qa_annoy.model')

True

In [ ]:
u = AnnoyIndex(300, 'angular')
u.load('qa_annoy.model') # super fast, will just mmap the file

True

In [5]:
print(u.get_nns_by_item(0, 2))

[0, 962601]


In [6]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=8).fit(X, df.answer)

In [19]:
outfile = open("qa_KNN_model.pcl",'wb')

pickle.dump(neigh, outfile)
outfile.close()

# Тесты

In [21]:
typical_phrases = ["Как дела?",
                   "Сколько мне лет?",
                   "Расскажи анекдот",
                   "Как понравиться человеку?",
                   "Как правильно намекнуть на то, что хочешь шубу?",
                   "Нравственно ли заниматься сексом до брака?",
                   "Расскажи мне о своей первой любви"]

In [22]:
for phrase in typical_phrases:
    print("---", phrase, "---")
    print()
    print(">>> annoy отвечает:")
    print(give_to_model(phrase, w2v, annoy_model))
    print()
    print(">>> KNN отвечает:")
    print(give_to_model(phrase, w2v, neigh))

--- Как дела? ---

>>> annoy отвечает:
заибись. 


>>> KNN отвечает:
Лучше всех!!! А у тебя?)))))))))). 

--- Сколько мне лет? ---

>>> annoy отвечает:
Порядка 30?. 


>>> KNN отвечает:
70 лет. 

--- Расскажи анекдот ---

>>> annoy отвечает:
а ты мне песню спой.. 


>>> KNN отвечает:
Колобок повесился. 

--- Как понравиться человеку? ---

>>> annoy отвечает:
не наглеть, не хамить, быть весел ым, дать понять, что она нравится, одеться опрятно, обаленно пахнуть и да... проследите за своими руками (в плане чистоты) и обувью.. 


>>> KNN отвечает:

Всем существом затрепетать 
над колокольчиком и мятой, 
и мотыльком уметь летать 
над синей бездной необъятной. 
 
Кружить над миром так легко 
в волнáх тончайшего потока, 
и замереть над васильком 
на грани - выдоха и вдоха… 
 
А зачем? 
 Если судьба сам влюбится, 
 а нет, сама лучше спрыгни с трамвая, легче будет. 
 Значит не по-пути.... 

--- Как правильно намекнуть на то, что хочешь шубу? ---

>>> annoy отвечает:
третий лишний. 


>>> KNN от

**По моей вскользь сделанной субъективной человеческой оценики, KNN и annoy показывают примерно одинаковое качество подбора ответа** 

Единственное, KNN обучается несколько секунд, а annoy - минут 20 и я не занимался оптимизацией модели.

# идеи и примечания

- Очистить ответы от HTML тегов, можно и удалить строги с разметкой